2

In [2]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
    --------------------------------------- 0.5/38.8 MB 5.6 MB/s eta 0:00:07
    --------------------------------------- 0.8/38.8 MB 2.1 MB/s eta 0:00:19
   - -------------------------------------- 1.0/38.8 MB 1.5 MB/s eta 0:00:25
   - -------------------------------------- 1.3/38.8 MB 1.4 MB/s eta 0:00:28
   - -------------------------------------- 1.3/38.8 MB 1.4 MB/s eta 0:00:28
   - -------------------------------------- 1.3/38.8 MB 1.4 MB/s eta 0:00:28
   - -------------------------------------- 1.6/38.8 MB 998.3 kB/s eta 0:00:38
   - -------------------------------------- 1.6/38.8 MB 998.3 kB/s eta 0:00:38
   - -------------------------------------- 1.8/38.8 MB 931.8 kB/s eta 0:00:40
   -- ------------------------------------- 2.4/38.8 MB 1.0 MB/s eta 0:00:36
   -- ------------------------------------- 2.6/38.8 MB 1.0 MB/s eta 0:00:35
   -- ------------------------------------- 2.9/38.8 MB 1.1 MB/s eta 0:00:34


In [11]:
import cv2 as cv
import numpy as np
from IPython.display import clear_output

# YOLO 모델 로드 함수
def construct_yolo3():
    with open('./yolov3/coco_names.txt', 'r') as f:
        class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('./yolov3/yolov3.weights', './yolov3/yolov3.cfg')
    layer_names = model.getLayerNames()
    output_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]
    return model, output_layers, class_names

# YOLO 초기화
model, out_layer, class_names = construct_yolo3()

# 객체 감지 함수
def yolo_detect(img):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0 / 256, (416, 416), (0, 0, 0), swapRB=True)
    model.setInput(test_img)
    output3 = model.forward(out_layer)

    boxes, confidences, class_ids = [], [], []

    for output in output3:
        for prediction in output:
            scores = prediction[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # 신뢰도가 50% 이상인 경우
                center_x, center_y = int(prediction[0] * width), int(prediction[1] * height)
                w, h = int(prediction[2] * width), int(prediction[3] * height)
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    objects = [boxes[i] + [confidences[i]] + [class_ids[i]] for i in range(len(boxes)) if i in indices]
    return objects

# 비디오 파일 경로
video_path = './sample.mp4'

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # 비디오가 끝나면 루프 종료
        frame = cv.resize(frame,(640,480))
        # 객체 감지
        objects = yolo_detect(frame)

        # 감지된 객체에 대해 바운딩 박스 및 레이블 그리기
        for obj in objects:
            x, y, w, h, conf, id = obj
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = f'{class_names[id]} {conf:.2f}'
            cv.putText(frame, text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 프레임 표시
        clear_output(wait=True)
        # cv2_imshow(frame)
        cv.imshow('object detection',frame)

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

# 비디오 캡처 객체 해제
cap.release()
cv.destroyAllWindows()

In [14]:
cap = cv.VideoCapture(0)
while True:
    ret, frame = cap.read()
    cv.imshow("webcam video", frame)
        # 'q' 키를 누르면 종료
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
# 비디오 캡처 객체 해제
cap.release()
cv.destroyAllWindows()    

In [15]:
# web cam 을 이용한 detect

import cv2 as cv
import numpy as np
from IPython.display import clear_output

# YOLO 모델 로드 함수
def construct_yolo3():
    with open('./yolov3/coco_names.txt', 'r') as f:
        class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('./yolov3/yolov3.weights', './yolov3/yolov3.cfg')
    layer_names = model.getLayerNames()
    output_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]
    return model, output_layers, class_names

# YOLO 초기화
model, out_layer, class_names = construct_yolo3()

# 객체 감지 함수
def yolo_detect(img):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0 / 256, (416, 416), (0, 0, 0), swapRB=True)
    model.setInput(test_img)
    output3 = model.forward(out_layer)

    boxes, confidences, class_ids = [], [], []

    for output in output3:
        for prediction in output:
            scores = prediction[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # 신뢰도가 50% 이상인 경우
                center_x, center_y = int(prediction[0] * width), int(prediction[1] * height)
                w, h = int(prediction[2] * width), int(prediction[3] * height)
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    objects = [boxes[i] + [confidences[i]] + [class_ids[i]] for i in range(len(boxes)) if i in indices]
    return objects

# 비디오 파일 경로
# video_path = './sample.mp4'

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # 비디오가 끝나면 루프 종료
        frame = cv.resize(frame,(640,480))
        # 객체 감지
        objects = yolo_detect(frame)

        # 감지된 객체에 대해 바운딩 박스 및 레이블 그리기
        for obj in objects:
            x, y, w, h, conf, id = obj
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = f'{class_names[id]} {conf:.2f}'
            cv.putText(frame, text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 프레임 표시
        clear_output(wait=True)
        # cv2_imshow(frame)
        cv.imshow('object detection',frame)

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

# 비디오 캡처 객체 해제
cap.release()
cv.destroyAllWindows()